***
<font size=6 color='blue'>Bulk Import from XML</font>   
***  

**Notebook Scope:**  
This notebook includes code to read in a series of XML files, combine them into a series of dataframes and export to CSV. The XML files include data on Congressional Bills from the 118th Congress, downloaded from the [ProPublica Data Store](https://www.propublica.org/datastore/dataset/congressional-data-bulk-legislation-bills).

**Output:**  
CSV Files
***  

# Notebook Setup
***

In [92]:
# Import libraries
import pathlib
import os
import datetime
import pandas as pd
import lxml.etree as ET
from bs4 import BeautifulSoup

In [2]:
# Define pandas options and defaults
#pd.options_display_max_colwidth = 30
def_style = [{'selector': 'td', 'props': 'white-space: nowrap; max-width: 300px; text-overflow: ellipsis; overflow: hidden;'}]

***
# Import XML Data
***

In [3]:
# Create dataframes
bills_df = pd.DataFrame()
committees_df = pd.DataFrame()
related_bills_df = pd.DataFrame()
actions_df = pd.DataFrame()
sponsors_df = pd.DataFrame()
cosponsors_df = pd.DataFrame()
subjects_df = pd.DataFrame()
summaries_df = pd.DataFrame()
titles_df = pd.DataFrame()

In [4]:
# Create a cross reference between elements and the df they will be stored in
dfs_dict = {'committees': committees_df, 'relatedBills': related_bills_df, 'actions': actions_df,
            'sponsors': sponsors_df, 'cosponsors': cosponsors_df, 'subjects': subjects_df,
            'summaries': summaries_df, 'titles': titles_df}

In [5]:
# Process multi element nodes
def proc_multi_nodes(parent, unique_id, elems, df, dup_tag=''):
    # Create a list of element tags contained in the elems object passed to the function
    tags = [x.tag for x in elems]
    
    # If there are multiple elements, but they all have the same tag, we will want to create a new row for each
    if len(set(tags)) == 1 and len(tags) > 1:
        dup_tag = parent + tags[0]
        
    for item in elems:
        if parent == '' or parent + item.tag == dup_tag:
            i = len(df)
            df.at[i, 'id'] = unique_id
        elif item.tag == 'subcommittees':
            return
        else:
            i = len(df) - 1
        if item.text is not None and item.text.strip() != '':
            if '<' in item.text:
                df.at[i, parent + item.tag] = BeautifulSoup(item.text, 'lxml').text
            else:
                df.at[i, parent + item.tag] = item.text
        proc_multi_nodes(parent + item.tag, unique_id, item, df, dup_tag)

In [20]:
# Create an iterable object containing all files by type - repeat in batches to reduce run time
path = '_data/sres/'
file_name ='fdsys_billstatus.xml' 
base_dir = pathlib.Path(path)
files_iter = base_dir.rglob(file_name)

In [21]:
# Read file and process
for file in files_iter:
    # Capture unique ID based on the files immediate parent directory
    right = str(file)[:-21].rfind('\\') + 1
    unique_id = str(file)[right:-21]

    # Read and parse the XML
    tree = ET.parse(file)
    bill = tree.find('bill')
    i = len(bills_df)

    # Procees each top level element in the current file
    for elem in bill:
        if elem.tag in dfs_dict.keys():
            proc_multi_nodes('', unique_id, elem, dfs_dict[elem.tag])
        elif elem.tag == 'policyArea':
            pa = elem.findall('name')
            bills_df.at[i, elem.tag] = pa[0].text
        else:
            if elem.text is not None and elem.text.strip() != '':
                bills_df.at[i, elem.tag] = elem.text

***
**<font color='blue'>Validation Check - File Count</font>**  
hr:       8,391  
s:        4,334  
hres:     1,226  
sres:       686  
hjres:      139  
hconres:    107  
sjres:       78  
sconres:     36  
**Total:   14,997** 
***

In [22]:
# Check rows loaded
bills_df['type'].value_counts()

type
HR         8391
S          4334
HRES       1226
SRES        686
HJRES       139
HCONRES     107
SJRES        78
SCONRES      36
Name: count, dtype: int64

***
# Save Temp Files
<font color='red'>Delete when finished</font>
***

In [23]:
# Save loaded data to CSV files to shorten processing on subsequent iterations of code cleanup
bills_df.to_csv('bills.csv', index=False)
committees_df.to_csv('committees.csv', index=False)
related_bills_df.to_csv('related_bills.csv', index=False)
actions_df.to_csv('actions.csv', index=False)
sponsors_df.to_csv('sponsors.csv', index=False)
cosponsors_df.to_csv('cosponsors.csv', index=False)
subjects_df.to_csv('subjects.csv', index=False)
summaries_df.to_csv('summaries.csv', index=False)
titles_df.to_csv('titles.csv', index=False)

In [ ]:
bills_df = pd.read_csv('bills.csv')
committees_df = pd.read_csv('committees.csv')
related_bills_df = pd.read_csv('related_bills.csv')
actions_df = pd.read_csv('actions.csv')
sponsors_df = pd.read_csv('sponsors.csv')
cosponsors_df = pd.read_csv('cosponsors.csv')
subjects_df = pd.read_csv('subjects.csv')
summaries_df = pd.read_csv('summaries.csv')
titles_df = pd.read_csv('titles.csv')

***
# Cleanup Data
***

## Bills
***

In [24]:
# Review bills_df
bills_df.head().style.set_table_styles(def_style)

,number,updateDate,updateDateIncludingText,originChamber,originChamberCode,type,introducedDate,congress,policyArea,title,constitutionalAuthorityStatementText
0,1,2024-04-17T23:49:12Z,2024-04-17T23:49:12Z,House,H,HCONRES,2023-01-09,118,Congress,Regarding consent to assemble outside the seat of government.,nan
1,10,2024-06-11T15:49:42Z,2024-06-11T15:49:42Z,House,H,HCONRES,2023-01-25,118,International Affairs,"Expressing the sense of Congress that the United States should resume normal diplomatic relations with Taiwan, negotiate a bilateral free trade agreement with Taiwan, and support Taiwan's membership in international organizations.",nan
2,100,2024-06-11T15:43:47Z,2024-06-11T15:43:47Z,House,H,HCONRES,2024-03-22,118,Congress,Directing the Clerk of the House of Representatives to make a correction in the enrollment of H.R. 2882.,nan
3,101,2024-06-11T15:43:56Z,2024-06-11T15:43:56Z,House,H,HCONRES,2024-03-22,118,"Civil Rights and Liberties, Minority Issues","Recognizing the need to improve physical access to many federally funded facilities for all people of the United States, particularly people with disabilities.",nan
4,102,2024-06-11T15:44:36Z,2024-06-11T15:44:36Z,House,H,HCONRES,2024-03-29,118,"Civil Rights and Liberties, Minority Issues",Supporting the goals and ideals of International Transgender Day of Visibility.,nan


In [25]:
# Review shape of dataframe
bills_df.shape

(14997, 11)

In [26]:
# Delete duplicates and nan columns
bills_df.drop_duplicates(inplace=True)
bills_df.dropna(axis=1, how='all', inplace=True)

In [27]:
# Review columns with NaN content
bills_df.isna().sum()

number                                     0
updateDate                                 0
updateDateIncludingText                    0
originChamber                              0
originChamberCode                         17
type                                       0
introducedDate                             0
congress                                   0
policyArea                               868
title                                      0
constitutionalAuthorityStatementText    6641
dtype: int64

In [28]:
# Review nan values in originChamberCode
bills_df[bills_df['originChamberCode'].isna()].style.set_table_styles(def_style)

,number,updateDate,updateDateIncludingText,originChamber,originChamberCode,type,introducedDate,congress,policyArea,title,constitutionalAuthorityStatementText
11226,2226,2023-08-25T00:00:30Z,2023-08-25T00:00:30Z,Senate,nan,S,2023-07-11,118,Armed Forces and National Security,National Defense Authorization Act for Fiscal Year 2024,nan
14618,376,2023-12-15T16:05:00Z,2023-12-15T15:24:23Z,Senate,nan,SRES,2023-09-27,118,Congress,Senate Dress Code Resolution,nan
14655,409,2023-12-15T16:05:04Z,2023-12-15T15:24:23Z,Senate,nan,SRES,2023-10-17,118,Congress,"A resolution authorizing the use of the atrium in the Philip A. Hart Senate Office Building for a Bipawtisan Howl-o-ween Dog Pawrade on October 31, 2023, from 4:00 to 6:00 p.m.",nan
14658,411,2023-12-15T16:05:04Z,2023-12-15T15:24:23Z,Senate,nan,SRES,2023-10-17,118,Congress,"A resolution to constitute the majority party's membership on certain committees for the One Hundred Eighteenth Congress, or until their successors are chosen.",nan
14662,415,2023-12-15T16:05:05Z,2023-12-15T15:24:23Z,Senate,nan,SRES,2023-10-18,118,Congress,A resolution to authorize testimony and representation in United States v. Samsel.,nan
14663,416,2023-12-15T16:05:05Z,2023-12-15T15:24:23Z,Senate,nan,SRES,2023-10-18,118,Congress,A resolution to authorize testimony and representation in United States v. Sullivan.,nan
14666,419,2023-12-15T16:05:05Z,2023-12-15T16:05:05Z,Senate,nan,SRES,2023-10-19,118,Public Lands and Natural Resources,"A resolution designating October 23, 2023, as ""National Marine Sanctuary Day"".",nan
14668,420,2023-12-15T16:05:05Z,2023-12-15T15:24:23Z,Senate,nan,SRES,2023-10-19,118,Congress,A resolution designating Betty K. Koed as Historian Emerita of the United States Senate.,nan
14673,425,2023-12-15T16:05:06Z,2023-12-15T16:05:06Z,Senate,nan,SRES,2023-10-24,118,"Arts, Culture, Religion","A resolution expressing the support of the Senate for the designation of ""Public Radio Music Day"" and deep appreciation for the role of public radio music stations in serving listeners, musicians, and hundreds of communities in the United States.",nan
14676,428,2023-12-15T16:05:06Z,2023-12-15T15:24:23Z,Senate,nan,SRES,2023-10-25,118,Sports and Recreation,"A resolution recognizing women's collegiate athletics and the record-setting Volleyball Day in Nebraska event on August 30, 2023.",nan


In [29]:
# Set missing values to 'S' for originChamberCode
bills_df['originChamberCode'] = bills_df['originChamberCode'].fillna('S')

In [30]:
# Set missing values to '' for policyArea and constitutionalAuthorityStatementText
bills_df['policyArea'] = bills_df['policyArea'].fillna('')
bills_df['constitutionalAuthorityStatementText'] = bills_df['constitutionalAuthorityStatementText'].fillna('')

In [31]:
# Review datatypes
bills_df.dtypes

number                                  object
updateDate                              object
updateDateIncludingText                 object
originChamber                           object
originChamberCode                       object
type                                    object
introducedDate                          object
congress                                object
policyArea                              object
title                                   object
constitutionalAuthorityStatementText    object
dtype: object

In [32]:
# Covert datatypes
num_cols = ['number', 'congress']
bills_df[num_cols] = bills_df[num_cols].astype('int')
bills_df['updateDate'] = pd.to_datetime(bills_df['updateDate'])
bills_df['updateDateIncludingText'] = pd.to_datetime(bills_df['updateDateIncludingText'])
bills_df['introducedDate'] = pd.to_datetime(bills_df['introducedDate'])

In [33]:
# Add id column
bills_df.insert(0, 'id', bills_df['type'] + bills_df['number'].astype('str'))

In [34]:
# Review number column data
print('Range of number values:')
print(f'{bills_df["number"].min()} - {bills_df["number"].max()}')

Range of number values:
1 - 8400


In [35]:
# Review date range for updateDate
print('Range of updateDate values:')
print(f'{bills_df["updateDate"].min().date()} to {bills_df["updateDate"].max().date()}')

Range of updateDate values:
2023-08-25 to 2024-06-15


In [36]:
# Review date range for updateDateIncludingText
print('Range of updateDateIncludingText values:')
print(f'{bills_df["updateDateIncludingText"].min().date()} to {bills_df["updateDateIncludingText"].max().date()}')

Range of updateDateIncludingText values:
2023-08-25 to 2024-06-15


In [37]:
# Review originChamber data
bills_df['originChamber'].unique()

array(['House', 'Senate'], dtype=object)

In [38]:
# Reveiw originChamberCode data
bills_df['originChamberCode'].unique()

array(['H', 'S'], dtype=object)

In [39]:
# Review type data
bills_df['type'].unique()

array(['HCONRES', 'HJRES', 'HR', 'HRES', 'S', 'SCONRES', 'SJRES', 'SRES'],
      dtype=object)

In [40]:
# Review date range for introducedDate
print('Range of introducedDate values:')
print(f'{bills_df["introducedDate"].min().date()} to {bills_df["introducedDate"].max().date()}')

Range of introducedDate values:
2023-01-03 to 2024-05-14


In [41]:
# Review congress data
bills_df['congress'].unique()

array([118])

In [42]:
# Reivew policyArea data
bills_df['policyArea'].unique()

array(['Congress', 'International Affairs',
       'Civil Rights and Liberties, Minority Issues', '',
       'Crime and Law Enforcement',
       'Foreign Trade and International Finance',
       'Science, Technology, Communications', 'Energy',
       'Armed Forces and National Security', 'Labor and Employment',
       'Health', 'Government Operations and Politics', 'Taxation',
       'Environmental Protection', 'Economics and Public Finance',
       'Immigration', 'Sports and Recreation',
       'Finance and Financial Sector', 'Agriculture and Food', 'Commerce',
       'Families', 'Law', 'Education', 'Transportation and Public Works',
       'Animals', 'Native Americans',
       'Public Lands and Natural Resources', 'Emergency Management',
       'Social Welfare', 'Housing and Community Development',
       'Water Resources Development', 'Arts, Culture, Religion',
       'Social Sciences and History', 'Private Legislation'], dtype=object)

In [43]:
# Review title data
bills_df['title'].unique()

array(['Regarding consent to assemble outside the seat of government.',
       "Expressing the sense of Congress that the United States should resume normal diplomatic relations with Taiwan, negotiate a bilateral free trade agreement with Taiwan, and support Taiwan's membership in international organizations.",
       'Directing the Clerk of the House of Representatives to make a correction in the enrollment of H.R. 2882.',
       ...,
       'A resolution celebrating the extraordinary accomplishments and vital role of women business owners in the United States.',
       'A resolution expressing concern about economic and security conditions in Mexico and reaffirming the interest of the United States in mutually beneficial relations with Mexico based on shared interests on security, economic prosperity, and democratic values, and for other purposes.',
       'A resolution declaring March 8, 2023, as "National Emily Warner & Women Airline Pilots Day".'],
      dtype=object)

In [44]:
# Review constitutionalAuthorityStatementText data
bills_df['constitutionalAuthorityStatementText'].unique()

array(['',
       '<pre>[Congressional Record Volume 169, Number 7 (Monday, January 9, 2023)][House]From the Congressional Record Online through the Government Publishing Office [<a href="https://www.gpo.gov">www.gpo.gov</a>]By Mr. BIGGS:H.J. Res. 1.Congress has the power to enact this legislation pursuantto the following:Article 5[Page H114]</pre>',
       '<pre>[Congressional Record Volume 169, Number 7 (Monday, January 9, 2023)][House]From the Congressional Record Online through the Government Publishing Office [<a href="https://www.gpo.gov">www.gpo.gov</a>]By Mr. McCLINTOCK:H.J. Res. 10.Congress has the power to enact this legislation pursuantto the following:Article V[Page H114]</pre>',
       ...,
       '<pre>[Congressional Record Volume 169, Number 30 (Tuesday, February 14, 2023)][House]From the Congressional Record Online through the Government Publishing Office [<a href="https://www.gpo.gov">www.gpo.gov</a>]By Mr. GOOD of Virginia:H.R. 997.Congress has the power to enact this

In [45]:
# Preview final version of dataframe
bills_df.head().style.set_table_styles(def_style)

,id,number,updateDate,updateDateIncludingText,originChamber,originChamberCode,type,introducedDate,congress,policyArea,title,constitutionalAuthorityStatementText
0,HCONRES1,1,2024-04-17 23:49:12+00:00,2024-04-17 23:49:12+00:00,House,H,HCONRES,2023-01-09 00:00:00,118,Congress,Regarding consent to assemble outside the seat of government.,
1,HCONRES10,10,2024-06-11 15:49:42+00:00,2024-06-11 15:49:42+00:00,House,H,HCONRES,2023-01-25 00:00:00,118,International Affairs,"Expressing the sense of Congress that the United States should resume normal diplomatic relations with Taiwan, negotiate a bilateral free trade agreement with Taiwan, and support Taiwan's membership in international organizations.",
2,HCONRES100,100,2024-06-11 15:43:47+00:00,2024-06-11 15:43:47+00:00,House,H,HCONRES,2024-03-22 00:00:00,118,Congress,Directing the Clerk of the House of Representatives to make a correction in the enrollment of H.R. 2882.,
3,HCONRES101,101,2024-06-11 15:43:56+00:00,2024-06-11 15:43:56+00:00,House,H,HCONRES,2024-03-22 00:00:00,118,"Civil Rights and Liberties, Minority Issues","Recognizing the need to improve physical access to many federally funded facilities for all people of the United States, particularly people with disabilities.",
4,HCONRES102,102,2024-06-11 15:44:36+00:00,2024-06-11 15:44:36+00:00,House,H,HCONRES,2024-03-29 00:00:00,118,"Civil Rights and Liberties, Minority Issues",Supporting the goals and ideals of International Transgender Day of Visibility.,


***
## Committees
***

In [46]:
# Review committees_df
committees_df.head().style.set_table_styles(def_style)

,id,itemsystemCode,itemname,itemchamber,itemtype,itemactivitiesitemname,itemactivitiesitemdate
0,hconres10,hswm00,Ways and Means Committee,House,Standing,Referred to,2023-01-25T15:00:35Z
1,hconres10,hsfa00,Foreign Affairs Committee,House,Standing,Referred to,2023-01-25T15:00:30Z
2,hconres101,hsif00,Energy and Commerce Committee,House,Standing,nan,nan
3,hconres101,hsgo00,Oversight and Accountability Committee,House,Standing,Referred to,2024-03-22T13:00:25Z
4,hconres101,hspw00,Transportation and Infrastructure Committee,House,Standing,Referred to,2024-03-22T13:00:15Z


In [47]:
# Review shape of dataframe
committees_df.shape

(21515, 7)

In [48]:
# Delete duplicates and nan columns
committees_df.drop_duplicates(inplace=True)
committees_df.dropna(axis=1, how='all', inplace=True)

In [49]:
# Review column names
committees_df.columns

Index(['id', 'itemsystemCode', 'itemname', 'itemchamber', 'itemtype',
       'itemactivitiesitemname', 'itemactivitiesitemdate'],
      dtype='object')

In [50]:
# Update column names for clarity
committees_df.columns = ['id', 'systemCode', 'name', 'chamber', 'type', 'activityName', 'activityDate']

In [51]:
# Review columns with NaN content
committees_df.isna().sum()

id                 0
systemCode      3210
name            3210
chamber         3210
type            3210
activityName    5133
activityDate    5133
dtype: int64

In [52]:
# The nans in systemCode, name, chamber and type are due to the process we used to read in the rows. We will forward fill 
# from the last valid value, and then remove any rows that have a nan value for activityName and activityDate
col_list = ['systemCode', 'name', 'chamber', 'type']
for col in col_list:
    committees_df[col] = committees_df[col].ffill()
committees_df.dropna(how='all', subset=['activityName', 'activityDate'], inplace=True)

In [53]:
# Review data types
committees_df.dtypes

id              object
systemCode      object
name            object
chamber         object
type            object
activityName    object
activityDate    object
dtype: object

In [54]:
# Covert datatypes
committees_df['activityDate'] = pd.to_datetime(committees_df['activityDate'])

In [55]:
# Review id column
committees_df['id'].unique()

array(['hconres10', 'hconres101', 'hconres102', ..., 'sres97', 'sres98',
       'sres99'], dtype=object)

In [56]:
# Review systemCode data
committees_df['systemCode'].unique()

array(['hswm00', 'hsfa00', 'hsgo00', 'hspw00', 'hsju00', 'hsed00',
       'hsha00', 'ssju00', 'hsas00', 'ssfr00', 'hsba00', 'hsif00',
       'hsru00', 'hsii00', 'hsvr00', 'ssfi00', 'hsag00', 'ssga00',
       'hlig00', 'ssra00', 'hsbu00', 'hsap00', 'hssy00', 'sseg00',
       'ssbk00', 'sscm00', 'ssva00', 'slia00', 'ssev00', 'ssaf00',
       'hssm00', 'sssb00', 'hshm00', 'sshr00', 'ssbu00', 'hsso00',
       'ssap00', 'ssas00', 'slin00', 'spag00'], dtype=object)

In [57]:
# Review name data
committees_df['name'].unique()[:15]

array(['Ways and Means Committee', 'Foreign Affairs Committee',
       'Oversight and Accountability Committee',
       'Transportation and Infrastructure Committee',
       'Judiciary Committee', 'Education and the Workforce Committee',
       'Committee on House Administration', 'Armed Services Committee',
       'Foreign Relations Committee', 'Financial Services Committee',
       'Energy and Commerce Committee', 'Rules Committee',
       'Natural Resources Committee', "Veterans' Affairs Committee",
       'Finance Committee'], dtype=object)

In [58]:
# Review chamber data
committees_df['chamber'].unique()

array(['House', 'Senate'], dtype=object)

In [59]:
# Review type data
committees_df['type'].unique()

array(['Standing', 'Select', 'Other', 'Special'], dtype=object)

In [60]:
# Review activityName data
committees_df['activityName'].unique()

array(['Referred to', 'Markup by', 'Discharged from', 'Reported by',
       'Unknown', 'Hearings by', 'Reported original measure',
       'Re-Referred to'], dtype=object)

In [61]:
# Review date range for activityDate
print('Range of activityDate values:')
print(f'{committees_df["activityDate"].min().date()} to {committees_df["activityDate"].max().date()}')

Range of activityDate values:
2023-01-09 to 2024-06-13


In [62]:
# Preview final version of dataframe
committees_df.head().style.set_table_styles(def_style)

,id,systemCode,name,chamber,type,activityName,activityDate
0,hconres10,hswm00,Ways and Means Committee,House,Standing,Referred to,2023-01-25 15:00:35+00:00
1,hconres10,hsfa00,Foreign Affairs Committee,House,Standing,Referred to,2023-01-25 15:00:30+00:00
3,hconres101,hsgo00,Oversight and Accountability Committee,House,Standing,Referred to,2024-03-22 13:00:25+00:00
4,hconres101,hspw00,Transportation and Infrastructure Committee,House,Standing,Referred to,2024-03-22 13:00:15+00:00
5,hconres101,hsju00,Judiciary Committee,House,Standing,Referred to,2024-03-22 13:00:10+00:00


***
## Related Bills
***

In [63]:
# Review related_bills_df
related_bills_df.head().style.set_table_styles(def_style)

,id,itemtitle,itemcongress,itemnumber,itemtype,itemlatestActionactionDate,itemlatestActiontext,itemrelationshipDetailsitemtype,itemrelationshipDetailsitemidentifiedBy,itemlatestActionactionTime
0,hconres100,"Further Consolidated Appropriations Act, 2024",118,2882,HR,2024-03-23,Became Public Law No: 118-47.,Related bill,House,nan
1,hconres101,"A concurrent resolution recognizing the need to improve physical access to many federally funded facilities for all people of the United States, particularly people with disabilities.",118,31,SCONRES,2024-03-22,"Referred to the Committee on Health, Education, Labor, and Pensions.",Identical bill,CRS,nan
2,hconres102,A concurrent resolution supporting the goals and ideals of International Transgender Day of Visibility.,118,32,SCONRES,2024-03-22,Referred to the Committee on the Judiciary.,Identical bill,CRS,nan
3,hconres102,Supporting the goals and ideals of International Transgender Day of Visibility.,118,32,HCONRES,2023-03-30,Referred to the House Committee on the Judiciary.,Identical bill,CRS,nan
4,hconres102,A resolution supporting the goals and ideals of International Transgender Day of Visibility.,118,154,SRES,2023-03-30,Referred to the Committee on the Judiciary. (text: CR S1105),Identical bill,CRS,nan


In [64]:
# Review shape of dataframe
related_bills_df.shape

(11955, 10)

In [65]:
# Delete duplicates and nan columns
related_bills_df.drop_duplicates(inplace=True)
related_bills_df.dropna(axis=1, how='all', inplace=True)

In [66]:
# Review column names
related_bills_df.columns

Index(['id', 'itemtitle', 'itemcongress', 'itemnumber', 'itemtype',
       'itemlatestActionactionDate', 'itemlatestActiontext',
       'itemrelationshipDetailsitemtype',
       'itemrelationshipDetailsitemidentifiedBy',
       'itemlatestActionactionTime'],
      dtype='object')

In [67]:
# Cleanup column names for clarity
related_bills_df.columns = ['id', 'title', 'congress', 'number', 'type', 'latestActionDate', 'latestActionText',
                            'relationshipType', 'relationshipIdentifiedBy', 'latestActionTime']

In [68]:
# Review columns with NaN content
related_bills_df.isna().sum()

id                              0
title                         631
congress                      629
number                        629
type                          629
latestActionDate              632
latestActionText              632
relationshipType              411
relationshipIdentifiedBy      411
latestActionTime            11173
dtype: int64

In [69]:
# The nans in title, congress, number, type, latestActionDate, and latestActionText are due to the process we used 
# to read in the rows. We will forward fill from the last valid value, and then remove any rows that have a nan 
# value for relationshipType and relationshipIdentifiedBy
col_list = ['title', 'congress', 'number', 'type', 'latestActionDate', 'latestActionText']
for col in col_list:
    related_bills_df[col] = related_bills_df[col].ffill()
related_bills_df.dropna(how='all', subset=['relationshipType', 'relationshipIdentifiedBy'], inplace=True)

In [70]:
# Delete the latestActionTime column because it is rarely populated and provides an unneeded level of detail
related_bills_df.drop(columns='latestActionTime', inplace=True)

In [71]:
# Review data types
related_bills_df.dtypes

id                          object
title                       object
congress                    object
number                      object
type                        object
latestActionDate            object
latestActionText            object
relationshipType            object
relationshipIdentifiedBy    object
dtype: object

In [72]:
# Covert datatypes
num_cols = ['congress', 'number']
related_bills_df[num_cols] = related_bills_df[num_cols].astype('int')
related_bills_df['latestActionDate'] = pd.to_datetime(related_bills_df['latestActionDate'])

In [73]:
# Review id data
related_bills_df['id'].unique()

array(['hconres100', 'hconres101', 'hconres102', ..., 'sres95', 'sres98',
       'sres99'], dtype=object)

In [74]:
# Review title data
related_bills_df['title'].unique()

array(['Further Consolidated Appropriations Act, 2024',
       'A concurrent resolution recognizing the need to improve physical access to many federally funded facilities for all people of the United States, particularly people with disabilities.',
       'A concurrent resolution supporting the goals and ideals of International Transgender Day of Visibility.',
       ...,
       'Memorializing those impacted by and lost to the COVID-19 pandemic.',
       'Honoring the life of Dr. Paul Farmer by recognizing the duty of the Federal Government to adopt a 21st-century global health solidarity strategy and take actions to address past and ongoing harms that undermine the health and well-being of people around the world.',
       'Expressing support for the designation of March 8, 2023, as "National Emily Warner and Women Airline Pilots Day".'],
      dtype=object)

In [75]:
# Review congress data
related_bills_df['congress'].unique()

array([118, 117, 116])

In [76]:
# Review number column data
print('Range of number values:')
print(f'{related_bills_df["number"].min()} - {related_bills_df["number"].max()}')

Range of number values:
1 - 8919


In [77]:
# Review type data
related_bills_df['type'].unique()

array(['HR', 'SCONRES', 'HCONRES', 'SRES', 'S', 'HRES', 'HJRES', 'SJRES'],
      dtype=object)

In [78]:
# Review date range for latestActionDate
print('Range of latestActionDate values:')
print(f'{related_bills_df["latestActionDate"].min().date()} to {related_bills_df["latestActionDate"].max().date()}')

Range of latestActionDate values:
2019-01-08 to 2024-06-12


In [79]:
# Review latestAction data
related_bills_df['latestActionText'].unique()

array(['Became Public Law No: 118-47.',
       'Referred to the Committee on Health, Education, Labor, and Pensions.',
       'Referred to the Committee on the Judiciary.', ...,
       'Submitted in the Senate, considered, and agreed to without amendment and with a preamble by Unanimous Consent. (consideration: CR S960; text: CR S958)',
       'Resolution agreed to in Senate without amendment and with a preamble by Unanimous Consent. (consideration: CR S2866; text: 03/19/2024 CR S2444)',
       'Referred to the Committee on Foreign Relations. (text: CR S2282-2284)'],
      dtype=object)

In [80]:
# Review relationshipType data
related_bills_df['relationshipType'].unique()

array(['Related bill', 'Identical bill', 'Procedurally-related',
       'Related document', 'Text similarities'], dtype=object)

In [81]:
# Review relationshiopIdentifiedBy data
related_bills_df['relationshipIdentifiedBy'].unique()

array(['House', 'CRS', 'Senate'], dtype=object)

In [82]:
# Final review related_bills_df
related_bills_df.head().style.set_table_styles(def_style)

,id,title,congress,number,type,latestActionDate,latestActionText,relationshipType,relationshipIdentifiedBy
0,hconres100,"Further Consolidated Appropriations Act, 2024",118,2882,HR,2024-03-23 00:00:00,Became Public Law No: 118-47.,Related bill,House
1,hconres101,"A concurrent resolution recognizing the need to improve physical access to many federally funded facilities for all people of the United States, particularly people with disabilities.",118,31,SCONRES,2024-03-22 00:00:00,"Referred to the Committee on Health, Education, Labor, and Pensions.",Identical bill,CRS
2,hconres102,A concurrent resolution supporting the goals and ideals of International Transgender Day of Visibility.,118,32,SCONRES,2024-03-22 00:00:00,Referred to the Committee on the Judiciary.,Identical bill,CRS
3,hconres102,Supporting the goals and ideals of International Transgender Day of Visibility.,118,32,HCONRES,2023-03-30 00:00:00,Referred to the House Committee on the Judiciary.,Identical bill,CRS
4,hconres102,A resolution supporting the goals and ideals of International Transgender Day of Visibility.,118,154,SRES,2023-03-30 00:00:00,Referred to the Committee on the Judiciary. (text: CR S1105),Identical bill,CRS


***
## Actions
***

In [83]:
# Review actions_df
actions_df.head().style.set_table_styles(def_style)

,id,itemactionDate,itemsourceSystemname,itemtext,itemtype,itemactionTime,itemactionCode,itemsourceSystemcode,itemcommitteesitemsystemCode,itemcommitteesitemname,itemcalendarNumbercalendar,itemrecordedVotesrecordedVoterollNumber,itemrecordedVotesrecordedVoteurl,itemrecordedVotesrecordedVotechamber,itemrecordedVotesrecordedVotecongress,itemrecordedVotesrecordedVotedate,itemrecordedVotesrecordedVotesessionNumber
0,hconres1,2023-01-23,Senate,Received in the Senate.,IntroReferral,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,hconres1,2023-01-09,House floor actions,Motion to reconsider laid on the table Agreed to without objection.,Floor,19:11:24,H38310,2,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,hconres1,2023-01-09,House floor actions,On agreeing to the resolution Agreed to without objection.,Floor,19:11:18,H37100,2,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,hconres1,2023-01-09,Library of Congress,Passed/agreed to in House On agreeing to the resolution Agreed to without objection. (text: CR H73),Floor,19:11:18,8000,9,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,hconres1,2023-01-09,House floor actions,Considered as privileged matter.,Floor,19:10:38,H30000,2,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [84]:
# Review shape of dataframe
actions_df.shape

(65350, 17)

In [85]:
# Delete duplicates and nan columns
actions_df.drop_duplicates(inplace=True)
actions_df.dropna(axis=1, how='all', inplace=True)

In [86]:
# Review column names
actions_df.columns

Index(['id', 'itemactionDate', 'itemsourceSystemname', 'itemtext', 'itemtype',
       'itemactionTime', 'itemactionCode', 'itemsourceSystemcode',
       'itemcommitteesitemsystemCode', 'itemcommitteesitemname',
       'itemcalendarNumbercalendar', 'itemrecordedVotesrecordedVoterollNumber',
       'itemrecordedVotesrecordedVoteurl',
       'itemrecordedVotesrecordedVotechamber',
       'itemrecordedVotesrecordedVotecongress',
       'itemrecordedVotesrecordedVotedate',
       'itemrecordedVotesrecordedVotesessionNumber'],
      dtype='object')

In [87]:
# Cleanup column names for clarity
actions_df.columns = ['id', 'date', 'sourceSystemName', 'text', 'type', 'actionTime', 'actionCode', 'sourceSystemCode',
                      'committeeCode', 'committeeName', 'calendarNumber', 'voteRollNumber', 'voteURL',
                      'chamber', 'congress', 'voteDate', 'sessionNumber']

In [88]:
# Review columns with NaN content
actions_df.isna().sum()

id                      0
date                   24
sourceSystemName       24
text                   24
type                   24
actionTime          56561
actionCode          14888
sourceSystemCode     7972
committeeCode       36126
committeeName       36126
calendarNumber      63822
voteRollNumber      63096
voteURL             63096
chamber             63096
congress            63096
voteDate            63096
sessionNumber       63096
dtype: int64

In [89]:
# All actions are from the 118th Congress
actions_df['congress'] = actions_df['congress'].fillna(118)

In [90]:
# Forward fill date, sourceSystemName, text, and type
cols = ['date', 'sourceSystemName', 'text', 'type']
actions_df[cols] = actions_df[cols].ffill()

In [93]:
# Set missing numbers based on date. Session 1 = Jan 3 2023 to Jan 2 2024; Session 2 = Jan 3 2024 to Jan 2 2025
actions_df.loc[pd.to_datetime(actions_df['date']) < datetime.datetime(2024, 1, 3), 'sessionNumber'] = 1
actions_df.loc[pd.to_datetime(actions_df['date']) >= datetime.datetime(2024, 1, 3), 'sessionNumber'] = 2

***
<font color='red'>**Note:**</font>  
Manual checking confirms that some NaN values are valid - not all columns relate to all actions. Before any cleanup can be done on NaN values caused by the import process, additional research needs to be done on the data, which is out of scope at this time.
***

In [94]:
# Covert datatypes
num_cols = ['congress', 'sessionNumber']
actions_df[num_cols] = actions_df[num_cols].astype('int')
actions_df['date'] = pd.to_datetime(actions_df['date'])
actions_df['voteDate'] = pd.to_datetime(actions_df['voteDate'])

In [95]:
actions_df.dtypes

id                               object
date                     datetime64[ns]
sourceSystemName                 object
text                             object
type                             object
actionTime                       object
actionCode                       object
sourceSystemCode                 object
committeeCode                    object
committeeName                    object
calendarNumber                   object
voteRollNumber                   object
voteURL                          object
chamber                          object
congress                          int32
voteDate            datetime64[ns, UTC]
sessionNumber                     int32
dtype: object

In [96]:
# Review id data
actions_df['id'].unique()

array(['hconres1', 'hconres10', 'hconres100', ..., 'sres97', 'sres98',
       'sres99'], dtype=object)

In [97]:
# Review date range for date
print('Range of date values:')
print(f'{actions_df["date"].min().date()} to {actions_df["date"].max().date()}')

Range of date values:
2023-01-03 to 2024-06-13


In [98]:
# Reivew sourceSystemName data
actions_df['sourceSystemName'].unique()

array(['Senate', 'House floor actions', 'Library of Congress',
       'House committee actions'], dtype=object)

In [99]:
# Review text data
actions_df['text'].unique()

array(['Received in the Senate.',
       'Motion to reconsider laid on the table Agreed to without objection.',
       'On agreeing to the resolution Agreed to without objection.', ...,
       'Star Print ordered on the reported resolution.',
       'Placed on Senate Legislative Calendar under General Orders. Calendar No. 54.',
       'Referred to the Committee on Foreign Relations. (text: CR S718-719)'],
      dtype=object)

In [100]:
# Review type data
actions_df['type'].unique()

array(['IntroReferral', 'Floor', 'Committee', 'Calendars', 'Discharge',
       'President', 'BecameLaw', 'Veto', 'ResolvingDifferences',
       'NotUsed'], dtype=object)

In [101]:
# Review range of actionTime data
print('Range of actionTime values:')
print(f'{actions_df["actionTime"].dropna().min()} to ', end='')
print(f'{actions_df["actionTime"].dropna().max()}')

Range of actionTime values:
00:01:28 to 23:57:58


In [102]:
# Review actionCode data
actions_df['actionCode'].unique()[:20]

array([nan, 'H38310', 'H37100', '8000', 'H30000', 'Intro-H', '1000',
       'H11100', '17000', 'H30200', 'H37300', 'H8D000', 'H30300',
       'H12200', '5000', 'H12420', '9000', 'H35000', 'H30800', 'H37220'],
      dtype=object)

In [103]:
# Review sourceSystemCode
actions_df['sourceSystemCode'].unique()

array([nan, '2', '9', '1'], dtype=object)

In [104]:
# Review committeeCode
actions_df['committeeCode'].unique()[:20]

array([nan, 'hswm00', 'hsfa00', 'hsif14', 'hsgo00', 'hsif00', 'hspw00',
       'hsju00', 'hsed00', 'hsha00', 'ssju00', 'hsag16', 'hsag00',
       'hsif03', 'hsii06', 'hspw14', 'hsii00', 'hspw13', 'hsas00',
       'hsha06'], dtype=object)

In [105]:
# Review committeeName data
actions_df['committeeName'].unique()[:15]

array([nan, 'Ways and Means Committee', 'Foreign Affairs Committee',
       'Health Subcommittee', 'Oversight and Accountability Committee',
       'Energy and Commerce Committee',
       'Transportation and Infrastructure Committee',
       'Judiciary Committee', 'Education and the Workforce Committee',
       'Committee on House Administration',
       'General Farm Commodities, Risk Management, and Credit Subcommittee',
       'Agriculture Committee',
       'Energy, Climate and Grid Security Subcommittee',
       'Energy and Mineral Resources Subcommittee',
       'Railroads, Pipelines, and Hazardous Materials Subcommittee'],
      dtype=object)

In [106]:
# Review calendarNumber
actions_df['calendarNumber'].unique()[:20]

array([nan, 'H00007', 'H00009', 'H00018', 'H00022', 'H00065', 'U00003',
       'U00071', 'U00024', 'H00012', 'U00049', 'U00051', 'U00072',
       'U00073', 'U00224', 'U00203', 'U00277', 'U00017', 'U00240',
       'U00016'], dtype=object)

In [107]:
# Review range of voteRollNumber
print('Range of voteRollNumber values:')
print(f'{int(actions_df['voteRollNumber'].dropna().min())} to {int(actions_df['voteRollNumber'].dropna().max())}')

Range of voteRollNumber values:
10 to 99


In [108]:
# Review votURL data
actions_df['voteURL'].unique()[:10]

array([nan, 'https://clerk.house.gov/evs/2023/roll136.xml',
       'https://clerk.house.gov/evs/2023/roll162.xml',
       'https://clerk.house.gov/evs/2023/roll30.xml',
       'https://clerk.house.gov/evs/2023/roll201.xml',
       'https://clerk.house.gov/evs/2023/roll227.xml',
       'https://clerk.house.gov/evs/2023/roll338.xml',
       'https://clerk.house.gov/evs/2023/roll36.xml',
       'https://clerk.house.gov/evs/2024/roll097.xml',
       'https://clerk.house.gov/evs/2023/roll106.xml'], dtype=object)

In [109]:
# Review chamber data
actions_df['chamber'].unique()

array([nan, 'House', 'Senate'], dtype=object)

In [110]:
# Review congress data
actions_df['congress'].unique()

array([118])

In [111]:
# Review range of date for voteDate
print('Range of voteDate values:')
print(f'{actions_df["voteDate"].min().date()} to {actions_df["voteDate"].max().date()}')

Range of voteDate values:
2023-01-09 to 2024-06-12


In [112]:
# Review sessionNumber data
actions_df['sessionNumber'].unique()

array([1, 2])

In [113]:
# Final review of actions_df
actions_df.head().style.set_table_styles(def_style)

,id,date,sourceSystemName,text,type,actionTime,actionCode,sourceSystemCode,committeeCode,committeeName,calendarNumber,voteRollNumber,voteURL,chamber,congress,voteDate,sessionNumber
0,hconres1,2023-01-23 00:00:00,Senate,Received in the Senate.,IntroReferral,nan,nan,nan,nan,nan,nan,nan,nan,nan,118,NaT,1
1,hconres1,2023-01-09 00:00:00,House floor actions,Motion to reconsider laid on the table Agreed to without objection.,Floor,19:11:24,H38310,2,nan,nan,nan,nan,nan,nan,118,NaT,1
2,hconres1,2023-01-09 00:00:00,House floor actions,On agreeing to the resolution Agreed to without objection.,Floor,19:11:18,H37100,2,nan,nan,nan,nan,nan,nan,118,NaT,1
3,hconres1,2023-01-09 00:00:00,Library of Congress,Passed/agreed to in House On agreeing to the resolution Agreed to without objection. (text: CR H73),Floor,19:11:18,8000,9,nan,nan,nan,nan,nan,nan,118,NaT,1
4,hconres1,2023-01-09 00:00:00,House floor actions,Considered as privileged matter.,Floor,19:10:38,H30000,2,nan,nan,nan,nan,nan,nan,118,NaT,1


***
## Sponsors
***

In [114]:
# Review sponsors_df
sponsors_df.head().style.set_table_styles(def_style)

,id,itembioguideId,itemfullName,itemfirstName,itemlastName,itemparty,itemstate,itemdistrict,itemisByRequest,itemmiddleName
0,hconres1,C001053,"Rep. Cole, Tom [R-OK-4]",Tom,Cole,R,OK,4,N,nan
1,hconres10,T000165,"Rep. Tiffany, Thomas P. [R-WI-7]",Thomas,Tiffany,R,WI,7,N,P.
2,hconres100,D000600,"Rep. Diaz-Balart, Mario [R-FL-26]",Mario,Diaz-Balart,R,FL,26,N,nan
3,hconres101,H001081,"Rep. Hayes, Jahana [D-CT-5]",Jahana,Hayes,D,CT,5,N,nan
4,hconres102,J000305,"Rep. Jacobs, Sara [D-CA-51]",Sara,Jacobs,D,CA,51,N,nan


In [115]:
# Review shape of dataframe
sponsors_df.shape

(14997, 10)

In [116]:
# Delete duplicates and nan columns
sponsors_df.drop_duplicates(inplace=True)
sponsors_df.dropna(axis=1, how='all', inplace=True)

In [117]:
# Review column names
sponsors_df.columns

Index(['id', 'itembioguideId', 'itemfullName', 'itemfirstName', 'itemlastName',
       'itemparty', 'itemstate', 'itemdistrict', 'itemisByRequest',
       'itemmiddleName'],
      dtype='object')

In [118]:
# Cleanup column names for clarity
sponsors_df.columns = ['id', 'bioguideId', 'fullName', 'firstName', 'lastName', 'party', 'state', 
                       'district', 'byRequest', 'middleName']

In [119]:
# Review columns with NaN content
sponsors_df.isna().sum()

id                0
bioguideId        0
fullName          0
firstName         0
lastName          0
party             0
state             0
district       5475
byRequest         0
middleName    10986
dtype: int64

In [120]:
# Review data types
sponsors_df.dtypes

id            object
bioguideId    object
fullName      object
firstName     object
lastName      object
party         object
state         object
district      object
byRequest     object
middleName    object
dtype: object

In [121]:
# Review id data
sponsors_df['id'].unique()

array(['hconres1', 'hconres10', 'hconres100', ..., 'sres97', 'sres98',
       'sres99'], dtype=object)

In [122]:
# Review bioguideId data
sponsors_df['bioguideId'].unique()[:20]

array(['C001053', 'T000165', 'D000600', 'H001081', 'J000305', 'B000490',
       'C001055', 'E000235', 'P000619', 'S001176', 'C001039', 'W000809',
       'L000589', 'P000605', 'D000623', 'G000558', 'M001156', 'W000788',
       'B001297', 'G000578'], dtype=object)

In [123]:
# Review fullName data
sponsors_df['fullName'].unique()[:15]

array(['Rep. Cole, Tom [R-OK-4]', 'Rep. Tiffany, Thomas P. [R-WI-7]',
       'Rep. Diaz-Balart, Mario [R-FL-26]', 'Rep. Hayes, Jahana [D-CT-5]',
       'Rep. Jacobs, Sara [D-CA-51]', 'Rep. Bishop, Sanford D. [D-GA-2]',
       'Rep. Case, Ed [D-HI-1]', 'Rep. Ezell, Mike [R-MS-4]',
       'Rep. Peltola, Mary Sattler [D-AK-At Large]',
       'Rep. Scalise, Steve [R-LA-1]', 'Rep. Cammack, Kat [R-FL-3]',
       'Rep. Womack, Steve [R-AR-3]', 'Rep. Lesko, Debbie [R-AZ-8]',
       'Rep. Perry, Scott [R-PA-10]', 'Rep. DeSaulnier, Mark [D-CA-10]'],
      dtype=object)

In [124]:
# Review firstName data
sponsors_df['firstName'].unique()[:20]

array(['Tom', 'Thomas', 'Mario', 'Jahana', 'Sara', 'Sanford', 'Ed',
       'Mike', 'Mary', 'Steve', 'Kat', 'Debbie', 'Scott', 'Mark', 'Brett',
       'Patrick', 'Nikema', 'Ken', 'Matt', 'Lois'], dtype=object)

In [125]:
# Reivew lastName data
sponsors_df['lastName'].unique()[:20]

array(['Cole', 'Tiffany', 'Diaz-Balart', 'Hayes', 'Jacobs', 'Bishop',
       'Case', 'Ezell', 'Peltola', 'Scalise', 'Cammack', 'Womack',
       'Lesko', 'Perry', 'DeSaulnier', 'Guthrie', 'McHenry', 'Williams',
       'Buck', 'Gaetz'], dtype=object)

In [126]:
# Review party data
sponsors_df['party'].unique()

array(['R', 'D', 'I'], dtype=object)

In [127]:
# Review state data
sponsors_df['state'].unique()

array(['OK', 'WI', 'FL', 'CT', 'CA', 'GA', 'HI', 'MS', 'AK', 'LA', 'AR',
       'AZ', 'PA', 'KY', 'NC', 'CO', 'MA', 'TX', 'TN', 'IL', 'NE', 'NJ',
       'MN', 'OR', 'NY', 'MO', 'MD', 'NM', 'WA', 'VA', 'IA', 'WV', 'IN',
       'OH', 'KS', 'NH', 'MT', 'SC', 'MI', 'ID', 'ND', 'RI', 'AL', 'SD',
       'DC', 'VI', 'NV', 'VT', 'DE', 'PR', 'UT', 'GU', 'WY', 'ME', 'MP',
       'AS'], dtype=object)

In [128]:
# Review district data
sponsors_df['district'].unique()

array(['4', '7', '26', '5', '51', '2', '1', nan, '3', '8', '10', '22',
       '19', '9', '16', '32', '12', '6', '11', '23', '17', '15', '24',
       '35', '27', '13', '38', '30', '18', '25', '14', '46', '33', '44',
       '36', '21', '40', '45', '43', '52', '28', '31', '49', '47', '39',
       '29', '41', '50', '48', '20', '37', '34', '42'], dtype=object)

In [129]:
# Review byRequest data
sponsors_df['byRequest'].unique()

array(['N'], dtype=object)

In [130]:
# Review middleName data
sponsors_df['middleName'].unique()

array([nan, 'P.', 'D.', 'Sattler', 'C.', 'R.', 'M.', 'N.', 'Z.', 'H.',
       'Ann', 'Morgan', 'S.', 'A. "Rick"', 'J.', 'X.', 'K.', 'Elvira',
       'T.', 'W.', 'French', 'B.', 'Scott', 'E.', 'C. "Hank"', 'A.', 'L.',
       'Taylor', 'Luis', 'Drew', 'HOLMES', 'McMorris', 'Diaz',
       'L. "Buddy"', 'O.', 'G.', 'Davis', 'F.', 'I.', 'C. "Bobby"',
       'Gregory', 'Kilili Camacho', 'Coleman', 'Dutch', 'Gluesenkamp',
       'J. "Chuck"', 'Gay', 'Strong', 'Holmes', 'G. "Chuy"', 'Moore',
       'Roland'], dtype=object)

***
## Cosponsors
***

In [131]:
# Review cosponsors_df
cosponsors_df.head().style.set_table_styles(def_style)

,id,itembioguideId,itemfullName,itemfirstName,itemlastName,itemparty,itemstate,itemdistrict,itemsponsorshipDate,itemisOriginalCosponsor,itemmiddleName,itemsponsorshipWithdrawnDate
0,hconres10,P000605,"Rep. Perry, Scott [R-PA-10]",Scott,Perry,R,PA,10,2023-01-25,True,nan,nan
1,hconres10,D000032,"Rep. Donalds, Byron [R-FL-19]",Byron,Donalds,R,FL,19,2023-01-25,True,nan,nan
2,hconres10,S001135,"Rep. Steel, Michelle [R-CA-45]",Michelle,Steel,R,CA,45,2023-01-25,True,nan,nan
3,hconres10,N000190,"Rep. Norman, Ralph [R-SC-5]",Ralph,Norman,R,SC,5,2023-01-25,True,nan,nan
4,hconres10,O000175,"Rep. Ogles, Andrew [R-TN-5]",Andrew,Ogles,R,TN,5,2023-01-25,True,nan,nan


In [132]:
# Review shape of dataframe
cosponsors_df.shape

(166057, 12)

In [133]:
# Delete duplicates and nan columns
cosponsors_df.drop_duplicates(inplace=True)
cosponsors_df.dropna(axis=1, how='all', inplace=True)

In [134]:
# Review column names
cosponsors_df.columns

Index(['id', 'itembioguideId', 'itemfullName', 'itemfirstName', 'itemlastName',
       'itemparty', 'itemstate', 'itemdistrict', 'itemsponsorshipDate',
       'itemisOriginalCosponsor', 'itemmiddleName',
       'itemsponsorshipWithdrawnDate'],
      dtype='object')

In [135]:
# Cleanup column names for clarity
cosponsors_df.columns = ['id', 'bioguideId', 'fullName', 'firstName', 'lastName', 'party', 'state', 
                       'district', 'sponsorshipDate', 'isOriginalCosponsor', 'middleName', 'sponsorshipWithdrawnDate']

In [136]:
# Review columns with NaN content
cosponsors_df.isna().sum()

id                               0
bioguideId                       0
fullName                         0
firstName                        0
lastName                         0
party                            0
state                            0
district                     31588
sponsorshipDate                  0
isOriginalCosponsor              0
middleName                  112259
sponsorshipWithdrawnDate    165489
dtype: int64

In [137]:
# Review data types
cosponsors_df.dtypes

id                          object
bioguideId                  object
fullName                    object
firstName                   object
lastName                    object
party                       object
state                       object
district                    object
sponsorshipDate             object
isOriginalCosponsor         object
middleName                  object
sponsorshipWithdrawnDate    object
dtype: object

In [138]:
# Covert datatypes
cosponsors_df['sponsorshipDate'] = pd.to_datetime(cosponsors_df['sponsorshipDate'])
cosponsors_df['sponsorshipWithdrawnDate'] = pd.to_datetime(cosponsors_df['sponsorshipWithdrawnDate'])

In [139]:
# Review id data
cosponsors_df['id'].unique()

array(['hconres10', 'hconres101', 'hconres102', ..., 'sres97', 'sres98',
       'sres99'], dtype=object)

In [140]:
# Review bioguideId data
cosponsors_df['bioguideId'].unique()[:20]

array(['P000605', 'D000032', 'S001135', 'N000190', 'O000175', 'L000578',
       'O000086', 'G000595', 'W000814', 'L000266', 'M000317', 'D000616',
       'G000589', 'C001103', 'M000194', 'M001136', 'C001120', 'M001177',
       'M001159', 'M001212'], dtype=object)

In [141]:
# Review fullName data
cosponsors_df['fullName'].unique()[:15]

array(['Rep. Perry, Scott [R-PA-10]', 'Rep. Donalds, Byron [R-FL-19]',
       'Rep. Steel, Michelle [R-CA-45]', 'Rep. Norman, Ralph [R-SC-5]',
       'Rep. Ogles, Andrew [R-TN-5]', 'Rep. LaMalfa, Doug [R-CA-1]',
       'Rep. Owens, Burgess [R-UT-4]', 'Rep. Good, Bob [R-VA-5]',
       'Rep. Weber, Randy K., Sr. [R-TX-14]',
       'Rep. LaTurner, Jake [R-KS-2]',
       'Rep. Malliotakis, Nicole [R-NY-11]',
       'Rep. DesJarlais, Scott [R-TN-4]', 'Rep. Gooden, Lance [R-TX-5]',
       'Rep. Carter, Earl L. "Buddy" [R-GA-1]',
       'Rep. Mace, Nancy [R-SC-1]'], dtype=object)

In [142]:
# Review firstName data
cosponsors_df['firstName'].unique()[:20]

array(['Scott', 'Byron', 'Michelle', 'Ralph', 'Andrew', 'Doug', 'Burgess',
       'Bob', 'Randy', 'Jake', 'Nicole', 'Lance', 'Earl', 'Nancy', 'Lisa',
       'Dan', 'Tom', 'Cathy', 'Barry', 'W.'], dtype=object)

In [143]:
# Reivew lastName data
cosponsors_df['lastName'].unique()[:20]

array(['Perry', 'Donalds', 'Steel', 'Norman', 'Ogles', 'LaMalfa', 'Owens',
       'Good', 'Weber', 'LaTurner', 'Malliotakis', 'DesJarlais', 'Gooden',
       'Carter', 'Mace', 'McClain', 'Crenshaw', 'McClintock', 'Rodgers',
       'Moore'], dtype=object)

In [144]:
# Review party data
cosponsors_df['party'].unique()

array(['R', 'D', 'I'], dtype=object)

In [145]:
# Review state data
cosponsors_df['state'].unique()

array(['PA', 'FL', 'CA', 'SC', 'TN', 'UT', 'VA', 'TX', 'KS', 'NY', 'GA',
       'MI', 'WA', 'AL', 'AZ', 'CO', 'IA', 'OH', 'WI', 'NJ', 'AR', 'OR',
       'NC', 'DC', 'NV', 'IL', 'MA', 'HI', 'VT', 'MO', 'CT', 'MN', 'NM',
       'RI', 'KY', 'NH', 'MD', 'MS', 'OK', 'LA', 'PR', 'NE', 'IN', 'ME',
       'WY', 'AS', 'MT', 'MP', 'ID', 'SD', 'WV', 'AK', 'GU', 'VI', 'DE',
       'ND'], dtype=object)

In [146]:
# Review district data
cosponsors_df['district'].unique()

array(['10', '19', '45', '5', '1', '4', '14', '2', '11', '9', '17', '8',
       '36', '26', '12', '3', '6', '7', '25', '28', '0', '20', '16', '18',
       '15', '29', '24', '32', '44', '34', '47', '30', '39', '13', '41',
       '31', '21', '27', '38', '22', '33', '23', '51', '37', '50', '35',
       '52', '40', '48', '49', '46', '43', '42', nan], dtype=object)

In [147]:
# Review date range of sponsorshipDate data
print('Range of sponsorshipDate values:')
print(f'{cosponsors_df["sponsorshipDate"].min().date()} to {cosponsors_df["sponsorshipDate"].max().date()}')

Range of sponsorshipDate values:
2023-01-09 to 2024-06-13


In [148]:
# Review isOriginalSponsor data
cosponsors_df['isOriginalCosponsor'].unique()

array(['True', 'False'], dtype=object)

In [149]:
# Review middleName data
cosponsors_df['middleName'].unique()

array([nan, 'K.', 'L. "Buddy"', 'C.', 'McMorris', 'Gregory', 'M.', 'P.',
       'W.', 'J.', 'J. "Chuck"', 'H.', 'A.', 'G.', 'Holmes', 'D.', 'E.',
       'O.', 'N.', 'Z.', 'Diaz', 'G. "Chuy"', 'R.', 'Gay', 'S.', 'L.',
       'F.', 'B.', 'I.', 'Scott', 'T.', 'A. "Rick"', 'Taylor', 'French',
       'Coleman', 'Kilili Camacho', 'Ann', 'X.', 'Dutch', 'Davis',
       'Sattler', 'Strong', 'Morgan', 'HOLMES', 'C. "Hank"', 'Elvira',
       'Drew', 'Luis', 'C. "Bobby"', 'Gluesenkamp', 'A. Dutch', 'Moore',
       'Roland'], dtype=object)

In [150]:
# Review date range of sponsorshipWithdrawnDate data
print('Range of sponsorshipWithdrawnDate values:')
print(f'{cosponsors_df["sponsorshipWithdrawnDate"].min().date()} to {cosponsors_df["sponsorshipWithdrawnDate"].max().date()}')

Range of sponsorshipWithdrawnDate values:
2023-01-11 to 2024-05-17


***
## Subjects
***

In [151]:
# Review subjects_df
subjects_df.head().style.set_table_styles(def_style)

,id,legislativeSubjectsitemname,policyAreaname
0,hconres1,Congressional operations and organization,nan
1,hconres1,nan,Congress
2,hconres10,nan,nan
3,hconres10,nan,International Affairs
4,hconres100,nan,nan


In [152]:
# Review shape of dataframe
subjects_df.shape

(71482, 3)

In [153]:
# Delete duplicates and nan columns
subjects_df.drop_duplicates(inplace=True)
subjects_df.dropna(axis=1, how='all', inplace=True)

In [154]:
# Review column names
subjects_df.columns

Index(['id', 'legislativeSubjectsitemname', 'policyAreaname'], dtype='object')

In [155]:
# Cleanup column names for clarity
subjects_df.columns = ['id', 'legislativeSubject', 'policyArea']

In [156]:
# Review columns with NaN content
subjects_df.isna().sum()

id                        0
legislativeSubject    28177
policyArea            57353
dtype: int64

In [157]:
# Delete rows that have NaN values for both legislativeSubject and policyArea
subjects_df.dropna(how='all', subset=['legislativeSubject', 'policyArea'], inplace=True)

In [158]:
# Review datatypes
subjects_df.dtypes

id                    object
legislativeSubject    object
policyArea            object
dtype: object

In [159]:
# Review id data
subjects_df['id'].unique()[:20]

array(['hconres1', 'hconres10', 'hconres100', 'hconres101', 'hconres102',
       'hconres103', 'hconres104', 'hconres106', 'hconres11', 'hconres12',
       'hconres13', 'hconres14', 'hconres15', 'hconres16', 'hconres17',
       'hconres18', 'hconres19', 'hconres2', 'hconres20', 'hconres22'],
      dtype=object)

In [160]:
# Review legislativeSubject data
subjects_df['legislativeSubject'].unique()[:15]

array(['Congressional operations and organization', nan, 'Appropriations',
       'Legislative rules and procedure', 'Conflicts and wars',
       'Congressional tributes', 'Military history', 'U.S. Capitol',
       'U.S. history', "Veterans' organizations and recognition",
       'Law enforcement administration and funding',
       'Law enforcement officers',
       'Presidents and presidential powers, Vice Presidents',
       'Agricultural prices, subsidies, credit', 'Agricultural trade'],
      dtype=object)

In [161]:
# Review policyArea data
subjects_df['policyArea'].unique()[:15]

array([nan, 'Congress', 'International Affairs',
       'Civil Rights and Liberties, Minority Issues',
       'Crime and Law Enforcement',
       'Foreign Trade and International Finance',
       'Science, Technology, Communications', 'Energy',
       'Armed Forces and National Security', 'Labor and Employment',
       'Health', 'Government Operations and Politics', 'Taxation',
       'Environmental Protection', 'Economics and Public Finance'],
      dtype=object)

In [162]:
# Final review subjects_df
subjects_df.head().style.set_table_styles(def_style)

,id,legislativeSubject,policyArea
0,hconres1,Congressional operations and organization,nan
1,hconres1,nan,Congress
3,hconres10,nan,International Affairs
5,hconres100,Appropriations,nan
6,hconres100,Legislative rules and procedure,nan


***
## Summaries
***

In [163]:
# Review summaries_df
summaries_df.head().style.set_table_styles(def_style)

,id,summaryversionCode,summaryactionDate,summaryactionDesc,summaryupdateDate,summarytext
0,hconres1,00,2023-01-09,Introduced in House,2023-01-12T20:27:36Z,This concurrent resolution authorizes the Speaker of the House and the Majority Leader of the Senate to assemble the House and the Senate outside the District of Columbia whenever the public interest warrants it.
1,hconres1,53,2023-01-09,Passed House,2023-01-13T15:52:05Z,This concurrent resolution authorizes the Speaker of the House and the Majority Leader of the Senate to assemble the House and the Senate outside the District of Columbia whenever the public interest warrants it.
2,hconres10,00,2023-01-25,Introduced in House,2023-02-02T21:11:43Z,This concurrent resolution calls on the President to abandon the One China policy in favor of one that recognizes Taiwan as an independent country that is not a part of China. The resolution also urges the President to bolster diplomatic and economic relations between the United States and Taiwan through specified means.
3,hconres101,00,2024-03-22,Introduced in House,2024-04-22T10:25:37Z,This concurrent resolution recognizes that people with disabilities experience barriers to access every day and pledges to make universal and inclusive design a guiding principle for all infrastructure bills and projects.
4,hconres102,00,2024-03-29,Introduced in House,2024-06-03T14:38:14Z,This concurrent resolution supports the goals and ideals of International Transgender Day of Visibility.


In [164]:
# Review shape of dataframe
summaries_df.shape

(10231, 6)

In [165]:
# Delete duplicates and nan columns
summaries_df.drop_duplicates(inplace=True)
summaries_df.dropna(axis=1, how='all', inplace=True)

In [166]:
# Review column names
summaries_df.columns

Index(['id', 'summaryversionCode', 'summaryactionDate', 'summaryactionDesc',
       'summaryupdateDate', 'summarytext'],
      dtype='object')

In [167]:
# Cleanup column names for clarity
summaries_df.columns = ['id', 'versionCode', 'actionDate', 'actionDesc', 'updateDate', 'summaryText']

In [168]:
# Review columns with NaN content
summaries_df.isna().sum()

id             0
versionCode    0
actionDate     0
actionDesc     0
updateDate     0
summaryText    0
dtype: int64

In [169]:
# Review datatypes
summaries_df.dtypes

id             object
versionCode    object
actionDate     object
actionDesc     object
updateDate     object
summaryText    object
dtype: object

In [170]:
# Covert datatypes
summaries_df['versionCode'] = summaries_df['versionCode'].astype('int')
summaries_df['actionDate'] = pd.to_datetime(summaries_df['actionDate'])
summaries_df['updateDate'] = pd.to_datetime(summaries_df['updateDate'])

In [171]:
# Reivew id data
summaries_df['id'].unique()

array(['hconres1', 'hconres10', 'hconres101', ..., 'sres97', 'sres98',
       'sres99'], dtype=object)

In [172]:
# Review versionCode
summaries_df['versionCode'].unique()

array([ 0, 53, 55,  8,  7, 49,  9, 59, 74, 25])

In [173]:
# Review date range of actionDate data
print('Range of actionDate values:')
print(f'{summaries_df["actionDate"].min().date()} to {summaries_df["actionDate"].max().date()}')

Range of actionDate values:
2023-01-03 to 2024-05-21


In [174]:
# Review actionDesc data
summaries_df['actionDesc'].unique()

array(['Introduced in House', 'Passed House', 'Passed Senate',
       'Reported to House, Part I', 'Reported to House', 'Public Law',
       'Reported to House, Part II', 'House agreed to Senate amendment',
       'Senate agreed to House amendment', 'Introduced in Senate',
       'Reported to Senate'], dtype=object)

In [175]:
# Review date range of updateDate data
print('Range of updateDate values:')
print(f'{summaries_df["updateDate"].min().date()} to {summaries_df["updateDate"].max().date()}')

Range of updateDate values:
2023-01-06 to 2024-06-14


In [176]:
# Review summaryText data
summaries_df['summaryText'].unique()[:5]

array(['This concurrent resolution authorizes the Speaker of the House and the Majority Leader of the Senate to assemble the House and the Senate outside the District of Columbia whenever the public interest warrants it.',
       'This concurrent resolution calls on the President to abandon the One China policy in favor of one that recognizes Taiwan as an independent country that is not a part of China. The resolution also urges the President to bolster diplomatic and economic relations between the United States and Taiwan through specified means.   ',
       'This concurrent resolution recognizes that people with disabilities experience barriers to access every day and pledges to make universal and inclusive design a guiding principle for all infrastructure bills and projects.',
       'This concurrent resolution supports the goals and ideals of International Transgender Day of Visibility.\xa0',
       'This concurrent resolution recognizes local law enforcement officers, extends grat

In [177]:
# Final review summaries_df
summaries_df.head().style.set_table_styles(def_style)

,id,versionCode,actionDate,actionDesc,updateDate,summaryText
0,hconres1,0,2023-01-09 00:00:00,Introduced in House,2023-01-12 20:27:36+00:00,This concurrent resolution authorizes the Speaker of the House and the Majority Leader of the Senate to assemble the House and the Senate outside the District of Columbia whenever the public interest warrants it.
1,hconres1,53,2023-01-09 00:00:00,Passed House,2023-01-13 15:52:05+00:00,This concurrent resolution authorizes the Speaker of the House and the Majority Leader of the Senate to assemble the House and the Senate outside the District of Columbia whenever the public interest warrants it.
2,hconres10,0,2023-01-25 00:00:00,Introduced in House,2023-02-02 21:11:43+00:00,This concurrent resolution calls on the President to abandon the One China policy in favor of one that recognizes Taiwan as an independent country that is not a part of China. The resolution also urges the President to bolster diplomatic and economic relations between the United States and Taiwan through specified means.
3,hconres101,0,2024-03-22 00:00:00,Introduced in House,2024-04-22 10:25:37+00:00,This concurrent resolution recognizes that people with disabilities experience barriers to access every day and pledges to make universal and inclusive design a guiding principle for all infrastructure bills and projects.
4,hconres102,0,2024-03-29 00:00:00,Introduced in House,2024-06-03 14:38:14+00:00,This concurrent resolution supports the goals and ideals of International Transgender Day of Visibility.


***
## Titles
***

In [178]:
# Review titles_df
titles_df.head().style.set_table_styles(def_style)

,id,itemtitleType,itemtitle,itemtitleTypeCode,itembillTextVersionName,itembillTextVersionCode,itemchamberCode,itemchamberName,itemsourceSystemcode,itemsourceSystemname
0,hconres1,Display Title,Regarding consent to assemble outside the seat of government.,nan,nan,nan,nan,nan,nan,nan
1,hconres1,Official Title as Introduced,Regarding consent to assemble outside the seat of government.,nan,nan,nan,nan,nan,nan,nan
2,hconres10,Display Title,"Expressing the sense of Congress that the United States should resume normal diplomatic relations with Taiwan, negotiate a bilateral free trade agreement with Taiwan, and support Taiwan's membership in international organizations.",45,nan,nan,nan,nan,nan,nan
3,hconres10,Official Title as Introduced,"Expressing the sense of Congress that the United States should resume normal diplomatic relations with Taiwan, negotiate a bilateral free trade agreement with Taiwan, and support Taiwan's membership in international organizations.",6,Introduced in House,IH,nan,nan,nan,nan
4,hconres100,Display Title,Directing the Clerk of the House of Representatives to make a correction in the enrollment of H.R. 2882.,45,nan,nan,nan,nan,nan,nan


In [179]:
# Review shape of dataframe
titles_df.shape

(45950, 10)

In [180]:
# Delete duplicates and nan columns
titles_df.drop_duplicates(inplace=True)
titles_df.dropna(axis=1, how='all', inplace=True)

In [181]:
# Review column names
titles_df.columns

Index(['id', 'itemtitleType', 'itemtitle', 'itemtitleTypeCode',
       'itembillTextVersionName', 'itembillTextVersionCode', 'itemchamberCode',
       'itemchamberName', 'itemsourceSystemcode', 'itemsourceSystemname'],
      dtype='object')

In [182]:
# Cleanup column names for clarity
titles_df.columns = ['id', 'titleType', 'title', 'titleTypeCode', 'billTextVersionName', 'billTextVersionCode', 
                     'chamberCode', 'chamberName', 'sourceSystemCode', 'sourceSystemName']

In [183]:
# Review columns with NaN content
titles_df.isna().sum()

id                         0
titleType                  0
title                      0
titleTypeCode           5954
billTextVersionName    16887
billTextVersionCode    16887
chamberCode            44382
chamberName            44382
sourceSystemCode       45945
sourceSystemName       45945
dtype: int64

In [184]:
# Set titleTypeCode based on titleType, based on roes that have values
type_codes_df = titles_df[~titles_df['titleTypeCode'].isna()][['titleType', 'titleTypeCode']].drop_duplicates()
type_codes_dict = type_codes_df.set_index('titleType').to_dict()
titles_df['titleTypeCode'] = titles_df['titleType'].map(type_codes_dict['titleTypeCode'])

In [185]:
# Review datatypes
titles_df.dtypes

id                     object
titleType              object
title                  object
titleTypeCode          object
billTextVersionName    object
billTextVersionCode    object
chamberCode            object
chamberName            object
sourceSystemCode       object
sourceSystemName       object
dtype: object

In [186]:
# Covert datatypes
titles_df['titleTypeCode'] = titles_df['titleTypeCode'].astype('int')

In [187]:
# Reivew id data
titles_df['id'].unique()

array(['hconres1', 'hconres10', 'hconres100', ..., 'sres97', 'sres98',
       'sres99'], dtype=object)

In [188]:
# Review titleType
titles_df['titleType'].unique()[:10]

array(['Display Title', 'Official Title as Introduced',
       'Short Title(s) as Introduced',
       'Official Titles as Amended by House',
       'Short Title(s) as Introduced for portions of this bill',
       'Short Title(s) as Passed House for portions of this bill',
       'Short Title(s) as Passed House',
       'Short Title(s) as Reported to House',
       'Short Titles as Passed Senate', 'Short Titles as Enacted'],
      dtype=object)

In [189]:
# Review title
titles_df['title'].unique()[:5]

array(['Regarding consent to assemble outside the seat of government.',
       "Expressing the sense of Congress that the United States should resume normal diplomatic relations with Taiwan, negotiate a bilateral free trade agreement with Taiwan, and support Taiwan's membership in international organizations.",
       'Directing the Clerk of the House of Representatives to make a correction in the enrollment of H.R. 2882.',
       'Recognizing the need to improve physical access to many federally funded facilities for all people of the United States, particularly people with disabilities.',
       'Supporting the goals and ideals of International Transgender Day of Visibility.'],
      dtype=object)

In [190]:
# Review titleTypeCode
titles_df['titleTypeCode'].unique()

array([ 45,   6, 101,   7, 106, 109, 104, 102,  18,  19, 103, 107,  17,
        27,  30,  26,  23,  14, 147,   9,  25,  22, 105,   8, 108, 110,
       250,  24, 253])

In [191]:
# Review billTextVersionName data
titles_df['billTextVersionName'].unique()

array([nan, 'Introduced in House', 'Engrossed in House',
       'Reported in House', 'Reported to Senate', 'Introduced in Senate',
       'Engrossed in Senate', 'Enrolled Bill'], dtype=object)

In [192]:
# Review billTextVersionCode data
titles_df['billTextVersionCode'].unique()

array([nan, 'IH', 'EH', 'RH', 'RS', 'IS', 'ES', 'ENR'], dtype=object)

In [193]:
# Review chamberCode data
titles_df['chamberCode'].unique()

array([nan, 'H', 'S', 'J'], dtype=object)

In [194]:
# Review chamberName data
titles_df['chamberName'].unique()

array([nan, 'House', 'Senate', 'Joint'], dtype=object)

In [195]:
# Review sourceSystemCode data
titles_df['sourceSystemCode'].unique()

array([nan, '9'], dtype=object)

In [196]:
# Review sourceSystemName data
titles_df['sourceSystemName'].unique()

array([nan, 'Library of Congress'], dtype=object)

In [197]:
# Final review titles_df
titles_df.head().style.set_table_styles(def_style)

,id,titleType,title,titleTypeCode,billTextVersionName,billTextVersionCode,chamberCode,chamberName,sourceSystemCode,sourceSystemName
0,hconres1,Display Title,Regarding consent to assemble outside the seat of government.,45,nan,nan,nan,nan,nan,nan
1,hconres1,Official Title as Introduced,Regarding consent to assemble outside the seat of government.,6,nan,nan,nan,nan,nan,nan
2,hconres10,Display Title,"Expressing the sense of Congress that the United States should resume normal diplomatic relations with Taiwan, negotiate a bilateral free trade agreement with Taiwan, and support Taiwan's membership in international organizations.",45,nan,nan,nan,nan,nan,nan
3,hconres10,Official Title as Introduced,"Expressing the sense of Congress that the United States should resume normal diplomatic relations with Taiwan, negotiate a bilateral free trade agreement with Taiwan, and support Taiwan's membership in international organizations.",6,Introduced in House,IH,nan,nan,nan,nan
4,hconres100,Display Title,Directing the Clerk of the House of Representatives to make a correction in the enrollment of H.R. 2882.,45,nan,nan,nan,nan,nan,nan


***
# Save to CSV
***

In [198]:
# Save loaded data to CSV files to shorten processing on subsequent iterations of code cleanup
bills_df.to_csv('bills.csv', index=False)
committees_df.to_csv('committees.csv', index=False)
related_bills_df.to_csv('related_bills.csv', index=False)
actions_df.to_csv('actions.csv', index=False)
sponsors_df.to_csv('sponsors.csv', index=False)
cosponsors_df.to_csv('cosponsors.csv', index=False)
subjects_df.to_csv('subjects.csv', index=False)
summaries_df.to_csv('summaries.csv', index=False)
titles_df.to_csv('titles.csv', index=False)

***
**End**
***